Download the data - after extracting features through a processing block - so we can train a machine learning model.

In [ ]:
import numpy as np
import requests

API_KEY = 'ei_f48a5b6d34abe5d231841baad96af1035721acb08136437b81d4ae33385aa2a8'

def download_data(url):
    response = requests.get(url, headers={'x-api-key': API_KEY})
    if response.status_code == 200:
        return response.content
    else:
        print(response.content)
        raise ConnectionError('Could not download data file')

X = download_data('https://studio.edgeimpulse.com/v1/api/41197/training/26/x')
Y = download_data('https://studio.edgeimpulse.com/v1/api/41197/training/26/y')


Store the data in a temporary file, and load it back through Numpy.

In [ ]:
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

Define our labels and split the data up in a test and training set:

In [ ]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "2.816", "5.1145", "5.883", "5.884", "5.962", "6.1215", "6.254", "6.4925", "10.971", "13.7005", "14.8135", "14.923", "15.237", "15.55", "18.046", "18.285", "20.299", "20.352", "20.66", "22.3925", "22.816", "25.4135", "27.124", "27.639", "27.931", "31.7735", "32.6745", "32.701", "33.4165", "35.2185", "36.0135", "36.9145", "37.736", "39.22", "42.188", "42.612", "42.9565", "42.983", "43.8045", "44.4935", "44.5465", "44.573", "44.785", "44.997", "46.9315", "47.0905", "47.9385", "49.634", "49.926", "50.032", "50.1645", "50.721", "50.8535", "51.622", "51.6485", "53.106", "53.371", "54.192", "54.4045", "54.749", "56.286", "58.406", "58.724", "58.83", "58.8565", "62.222", "62.722", "62.8315", "62.9375", "63.6265", "65.295", "65.4765", "65.587", "68.529", "68.8205", "72.928", "74.4385" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def get_reshape_function(reshape_to):
    def reshape(image, label):
        return tf.reshape(image, reshape_to), label
    return reshape

callbacks = []


Train the model:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization, TimeDistributed
from tensorflow.keras.optimizers import Adam

# model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', kernel_constraint=tf.keras.constraints.MaxNorm(1), padding='same'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model.add(Conv2D(16, kernel_size=3, activation='relu', kernel_constraint=tf.keras.constraints.MaxNorm(1), padding='same'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(classes, name='y_pred'))

# this controls the learning rate
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# This is only required for some input types, e.g. images
SPECIFIC_INPUT_SHAPE = (96, 96, 3)
train_dataset = train_dataset.map(get_reshape_function(SPECIFIC_INPUT_SHAPE), tf.data.experimental.AUTOTUNE)
validation_dataset = validation_dataset.map(get_reshape_function(SPECIFIC_INPUT_SHAPE), tf.data.experimental.AUTOTUNE)
# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)

# train the neural network
model.compile(loss='mean_squared_error', optimizer=opt)
model.fit(train_dataset, epochs=130, validation_data=validation_dataset, verbose=2, callbacks=callbacks)


In [ ]:
# Save the model to disk
model.save('saved_model')